In [ ]:
from google.colab import files

# Upload the .mat file
uploaded = files.upload()

# Now the file is available in the current working directory
mat_path = "Data.mat"  # Example file name, change to your file name

In [ ]:
uploaded = files.upload()
mat_path_gtm = "GTM.mat"

In [ ]:
# @title Default title text
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import models
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, cohen_kappa_score
import random
import cv2
import matplotlib.pyplot as plt
import scipy.io
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import scipy.io as sio
import time
from sklearn.metrics import mutual_info_score
from sklearn.svm import SVC
from torch.utils.data import Sampler
# --------------------------
# 1. Generate Synthetic HSI Data
# --------------------------

# Load HSI data and ground truth map
data = scipy.io.loadmat('Data.mat')
gtm = scipy.io.loadmat('GTM.mat')

hsi = data['Data']  # Hyperspectral image data
gt = gtm['GTM']  # Ground truth map

# Display one band of HSI data (e.g., Band 1)
plt.figure(figsize=(8, 6))
plt.imshow(hsi[:, :, 0], cmap='gray')
plt.colorbar()
plt.title("HSI Band 1")
plt.show()

# Display the GTM
plt.figure(figsize=(8, 6))
plt.imshow(gt, cmap='jet')  # 'jet' provides better visualization for categorical data
plt.colorbar()  # Add a color legend
plt.title("Ground Truth Map (GTM)")
plt.show()

# --------------------------
# 2. Patch Extraction and Preprocessing
# --------------------------
def extract_patch(hsi, center, patch_size=32):
    """Extract a cube patch from the HSI given the center and patch size."""
    h, w, _ = hsi.shape
    half = patch_size // 2
    c_i, c_j = center
    # Limit to image boundaries
    start_i = max(c_i - half, 0)
    end_i = min(c_i + half, h)
    start_j = max(c_j - half, 0)
    end_j = min(c_j + half, w)
    patch = hsi[start_i:end_i, start_j:end_j, :]
    # If patch size is less than required, pad it
    if patch.shape[0] < patch_size or patch.shape[1] < patch_size:
        patch = cv2.copyMakeBorder(patch,
                                   top=half - (c_i - start_i),
                                   bottom=half - (end_i - c_i),
                                   left=half - (c_j - start_j),
                                   right=half - (end_j - c_j),
                                   borderType=cv2.BORDER_REFLECT)
    return patch

def apply_pca(patch, n_components=3):
    """Apply PCA to reduce the spectral dimension of the patch to n_components."""
    h, w, bands = patch.shape
    patch_2d = patch.reshape(-1, bands)
    pca = PCA(n_components=n_components)
    patch_reduced = pca.fit_transform(patch_2d)
    patch_reduced = patch_reduced.reshape(h, w, n_components)
    # Normalize to [0, 1]
    patch_reduced = (patch_reduced - patch_reduced.min()) / (patch_reduced.max() - patch_reduced.min() + 1e-8)
    return patch_reduced


# --------------------------
# 3. Data Augmentation for Generating Two Views
# --------------------------
data_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomCrop(28),  # Crop to a smaller size
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ToTensor()
])

def generate_two_views(patch):
    """
    Generate two different augmented views from the input patch (numpy array).
    The patch is assumed to be PCA-reduced and then augmented.
    """
    patch_uint8 = (patch * 255).astype(np.uint8)
    view1 = data_transforms(patch_uint8)
    view2 = data_transforms(patch_uint8)
    return view1, view2

# --------------------------
# 4. Calculate Difficulty of Each Patch
# --------------------------


# Function to calculate difficulty for a single pixel based on the provided formula
def calculate_pixel_difficulty(H_i, alpha_i):
    # Calculate difficulty using the formula
    difficulty = np.sqrt(H_i**2 + (1 - np.abs((alpha_i - 60) / 60))**2)
    return difficulty

# Function to calculate difficulty for the entire patch
def calculate_patch_difficulty(patch):
    height, width, _ = patch.shape
    difficulties = []

    # Loop through every pixel in the patch
    for i in range(height):
        for j in range(width):
            # Get H_i from the first band (H_i is the value at the first band of the pixel)
            H_i = patch[i, j, 0]  # Value from the first band (H_i)

            # Get alpha_i from the second band (alpha_i is the value at the second band of the pixel)
            alpha_i = patch[i, j, 1]  # Value from the second band (α_i)

            # Calculate difficulty for this pixel
            pixel_difficulty = calculate_pixel_difficulty(H_i, alpha_i)
            difficulties.append(pixel_difficulty)

    # Calculate the average difficulty for the patch
    patch_difficulty = np.mean(difficulties)
    return patch_difficulty



# --------------------------
# 6. Self-Supervised Dataset Definition
# --------------------------

class HSISelfSupervisedFullDataset(Dataset):
    def __init__(self, hsi, patch_size=32):
        """
        Extract patches from the entire HSI image, one for each pixel.
        """
        self.hsi = hsi
        self.patch_size = patch_size
        self.height, self.width, _ = hsi.shape

    def __len__(self):
        # Return total number of pixels in the image
        return self.height * self.width

    def __getitem__(self, idx):
        # Convert the 1D index to 2D coordinates (i, j)
        i = idx // self.width
        j = idx % self.width
        center = (i, j)

        # Extract a patch centered at (i, j)
        patch = extract_patch(self.hsi, center, self.patch_size)
        # Apply PCA to reduce spectral bands to 3 for compatibility with EfficientNet-B0
        patch_pca = apply_pca(patch, n_components=3)
        # Generate two different augmented views of the patch
        view1, view2 = generate_two_views(patch_pca)
        return view1, view2


# --------------------------
# 7. Deep Curriculum Algorthm
# --------------------------

class CurriculumBatchSampler(Sampler):
    def __init__(self, difficulties, batch_size):
        """
        A sampler that generates batches ordered from easy to hard based on difficulty scores.

        Args:
            difficulties (list or array): Difficulty score for each sample (lower = easier).
            batch_size (int): Number of samples in each mini-batch.
        """
        self.difficulties = np.array(difficulties)
        self.batch_size = batch_size

        # Sort all sample indices based on difficulty (easy to hard)
        self.sorted_indices = np.argsort(self.difficulties)

    def __iter__(self):
        """
        Yield batches in curriculum learning order: easy to hard.
        """
        for i in range(0, len(self.sorted_indices), self.batch_size):
            batch = self.sorted_indices[i:i + self.batch_size]
            # Sort each batch internally by difficulty (just to be sure)
            batch = sorted(batch, key=lambda idx: self.difficulties[idx])
            yield batch

    def __len__(self):
        """
        Total number of batches.
        """
        return (len(self.sorted_indices) + self.batch_size - 1) // self.batch_size


# --------------------------
# 8. Define the ES2FL Model Based on EfficientNet-B0
# --------------------------

class ES2FLModel(nn.Module):
    def __init__(self):
        super(ES2FLModel, self).__init__()

        # Load EfficientNet-B0 from torch.hub (no pretrained weights)
        self.backbone = torch.hub.load('rwightman/gen-efficientnet-pytorch', 'efficientnet_b0', pretrained=False)

        # Extract the stem (initial conv + BN + activation)
        self.stem = nn.Sequential(
            self.backbone.conv_stem,
            self.backbone.bn1,
            self.backbone.act1
        )

        # Extract the first 10 MBConv blocks
        self.blocks = nn.Sequential(*self.backbone.blocks[:10])

    def forward(self, x):
        x = self.stem(x)        # Converts input from 3 channels → 32 channels
        x = self.blocks(x)      # Passes through first 10 MBConv blocks
        x = torch.flatten(x, start_dim=1)  # Flatten from [B, C, H, W] to [B, C*H*W]
        return x

# --------------------------
# 9. Self-Supervised Loss Function
# --------------------------
def self_supervised_loss(z_a, z_b, lambd=0.005):
    """
    Compute the self-supervised loss:
      L = sum_i (1 - C_ii)^2 + λ * sum_{i≠j} C_ij^2
    where C is the cross-correlation matrix between two feature sets.
    """
    batch_size, feat_dim = z_a.shape
    # Compute cross-correlation matrix
    c = torch.mm(z_a.T, z_b) / batch_size  # Shape: (feat_dim, feat_dim)

    # Diagonal loss: make diagonal values close to 1
    on_diag = torch.diagonal(c).add(-1).pow(2).sum()
    # Off-diagonal loss: make off-diagonal values close to 0
    off_diag = (c - torch.diag(torch.diagonal(c))).pow(2).sum()

    loss = on_diag + lambd * off_diag
    return loss

# --------------------------
# 10. Self-Supervised Training Function
# --------------------------


def train_with_cumulative_curriculum(patches, difficulties, batch_size, model, optimizer, device, epoch, total_epochs):
    import random
    model.train()

    # Shuffle indices at the beginning of each epoch
    indices = list(range(len(patches)))
    random.shuffle(indices)

    # Divide into random batches
    batches = [indices[i:i + batch_size] for i in range(0, len(indices), batch_size)]

    # Compute average difficulty for each batch
    batch_difficulties = [np.mean([difficulties[idx] for idx in batch]) for batch in batches]

    # Sort batches based on their average difficulty (ascending)
    sorted_batch_indices = np.argsort(batch_difficulties)

    cumulative_indices = []
    for iteration, batch_idx in enumerate(sorted_batch_indices, 1):
        # Add current batch to cumulative list
        cumulative_indices.extend(batches[batch_idx])

        # Prepare current cumulative batch
        selected_patches = [patches[i] for i in cumulative_indices]
        views1, views2 = zip(*selected_patches)
        views1, views2 = torch.stack(views1).to(device), torch.stack(views2).to(device)

        # Training step
        optimizer.zero_grad()
        z_a = model(views1)
        z_b = model(views2)
        loss = self_supervised_loss(z_a, z_b, lambd=0.005)
        loss.backward()
        optimizer.step()

        print(f"Epoch [{epoch}/{total_epochs}] Iteration [{iteration}/{len(batches)}] Loss: {loss.item():.4f}")

    return model


# --------------------------
# 11. Feature Extraction and Classification (Using Random Forest as Example)
# --------------------------
def extract_features(model, hsi, patch_size=32, centers=None, device='cuda'):
    """
    Extract deep features for specified pixel centers.
    centers: list of tuples (i, j)
    """
    model.eval()
    features_list = []
    with torch.no_grad():
        for center in centers:
            patch = extract_patch(hsi, center, patch_size)
            patch_pca = apply_pca(patch, n_components=3)
            # Convert to tensor and apply necessary transforms
            patch_tensor = transforms.ToTensor()( (patch_pca*255).astype(np.uint8) ).unsqueeze(0).to(device)
            feat = model(patch_tensor)
            feat = feat.cpu().numpy().flatten()
            features_list.append(feat)
    return np.array(features_list)

def extract_spectral_vector(hsi, center):
    """Return the spectral vector at the given center pixel."""
    i, j = center
    spectral_vec = hsi[i, j, :]
    return spectral_vec

def get_labeled_centers(gt, samples_per_class=5, num_classes=5, seed=42):
    """Randomly select labeled centers (pixels) for each class."""
    random.seed(seed)
    centers = []
    labels = []
    h, w = gt.shape
    for cls in range(num_classes):
        cls_indices = np.argwhere(gt == cls)
        selected = random.sample(cls_indices.tolist(), samples_per_class)
        for s in selected:
            centers.append(tuple(s))
            labels.append(cls)
    return centers, np.array(labels)

# --------------------------
# 12. Spectral Features
# --------------------------

def extract_raw_spectral_features(hsi_subset, centers):
    """
    Extract raw spectral features (direct pixel values) from the given HSI subset
    at specified spatial centers.

    Parameters:
        hsi_subset (np.ndarray): HSI data for one spectral group [H, W, bands].
        centers (list of tuples): List of (i, j) spatial coordinates.

    Returns:
        np.ndarray: Array of shape [num_samples, num_bands] with spectral vectors.
    """
    features = []
    for i, j in centers:
        features.append(hsi_subset[i, j, :])
    return np.array(features)


# --------------------------
# 13. Padding Array
# --------------------------

def apply_padding(features, patch=3):
    """
    Apply padding to the feature vector to convert it to a 2D matrix.

    features: 1D array of features (feature_dim,)
    patch_size: Size of the patch (3x3 for example, resulting in a 3x3 matrix)

    Returns: 2D matrix (patch_size x patch_size x feature_dim)
    """
    # Get the dimension of the feature vector
    feature_dim = features.shape[0]  # Should be 1024 for example

    # Initialize a zero matrix for the padded features
    padded_features = np.zeros((patch, patch, feature_dim))  # Initialize a zero matrix

    # Fill the center of the patch with the features
    padded_features[patch // 2, patch // 2, :] = features  # Place feature vector in the center
    return padded_features

# Applying padding on the combined features:
def apply_padding_to_features(features_list, patch=3):
    """
    Apply padding to all features in the list and return the padded features.

    features_list: List of feature vectors (N x feature_dim)
    patch_size: Size of the patch (3x3 for example)

    Returns: List of padded feature matrices (N x patch_size x patch_size x feature_dim)
    """
    padded_features_list = []

    for features in features_list:
        padded_features = apply_padding(features, patch)
        padded_features_list.append(padded_features)

    return np.array(padded_features_list)


# --------------------------------
# 14. Local Bainary Graph Algorithm
# --------------------------------


from scipy.spatial.distance import cdist
from scipy.linalg import eigh

def build_graph_matrix(patch_reshaped, k=4):
    """
    Constructs adjacency matrix A using k-nearest neighbors based on Euclidean distance.
    :param patch_reshaped: numpy array of shape (N, C), where N is number of pixels and C is number of spectral bands
    :param k: number of nearest neighbors
    :return: adjacency matrix A of shape (N, N)
    """
    N = patch_reshaped.shape[0]

    # Compute Euclidean distance between all pixel vectors
    distances = cdist(patch_reshaped, patch_reshaped, metric='euclidean')

    # For each pixel, keep only k nearest neighbors (excluding self)
    A = np.zeros((N, N))
    for i in range(N):
        idx = np.argsort(distances[i])[1:k+1]  # Skip self (0th index)
        A[i, idx] = 1
        A[idx, i] = 1  # Make it symmetric

    return A

def compute_degree_matrix(A):
    """
    Constructs diagonal degree matrix D from adjacency matrix A.
    :param A: adjacency matrix of shape (N, N)
    :return: degree matrix D of shape (N, N)
    """
    D = np.diag(np.sum(A, axis=1))
    return D

def compute_laplacian(D, A):
    """
    Computes unnormalized graph Laplacian matrix L = D - A.
    :param D: degree matrix
    :param A: adjacency matrix
    :return: Laplacian matrix L
    """
    return D - A

def compute_feature_fusion(patch, output_channels=40, k=4):
    """
    Applies graph-based feature fusion on a given patch.
    :param patch: numpy array of shape (H, W, C)
    :param output_channels: number of output fused spectral dimensions
    :param k: number of nearest neighbors in the graph
    :return: fused features of shape (H*W, output_channels)
    """
    H, W, C = patch.shape
    N = H * W

    # Step 1: Reshape patch to (N, C)
    X = patch.reshape(N, C)

    # Step 2: Construct graph adjacency matrix A
    A = build_graph_matrix(X, k=k)

    # Step 3: Compute degree matrix D
    D = compute_degree_matrix(A)

    # Step 4: Compute Laplacian matrix L = D - A
    L = compute_laplacian(D, A)

    # Step 5: Compute P = X^T * D * X
    P = X.T @ D @ X  # Shape: (C, C)

    # Regularization for numerical stability
    epsilon = 1e-5
    P += epsilon * np.eye(P.shape[0])

    # Step 6: Compute Q = X^T * L * X
    Q = X.T @ L @ X  # Shape: (C, C)

    # Step 7: Solve generalized eigenvalue problem Qw = λPw
    eigvals, eigvecs = eigh(Q, P)

    # Step 8: Select top 'output_channels' eigenvectors (with largest eigenvalues)
    sorted_idx = np.argsort(eigvals)[::-1]
    top_eigvecs = eigvecs[:, sorted_idx[:output_channels]]  # Shape: (C, output_channels)

    # Step 9: Multiply reshaped patch (N, C) with weight matrix (C, output_channels)
    fused_features = X @ top_eigvecs  # Resulting shape: (N, output_channels)

    return fused_features  # shape: (H*W, output_channels)

def batch_feature_fusion(patches, output_channels=40, k=4):
    """
    Applies graph-based feature fusion to a batch of patches.
    :param patches: numpy array of shape (N, H, W, C)
    :param output_channels: number of output spectral features
    :param k: number of neighbors for graph construction
    :return: numpy array of shape (N, H*W, output_channels)
    """
    N, H, W, C = patches.shape
    fused_batch = np.zeros((N, H * W, output_channels))

    for i in range(N):
        fused_batch[i] = compute_feature_fusion(patches[i], output_channels=output_channels, k=k)

    return fused_batch



# --------------------------
# 15. Main Function to Run the Entire Process
# --------------------------


def main():


    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Self-supervised training using full HSI
    print("ِPatch Ranking Uisng The Deep Curriculum Learning Model Has Been Started")
    ss_dataset = HSISelfSupervisedFullDataset(hsi, patch_size=32)

    # List of patches and their difficulty scores
    patches = []
    difficulties = []
    print(f" Number of Pathces: {len(ss_dataset)}")

    for idx in range(len(ss_dataset)):
        view1, view2 = ss_dataset[idx]
        patch = view1.numpy()  # For example, use view1 to compute difficulty
        difficulty = calculate_patch_difficulty(patch)
        # Print the calculated difficulty
        # print(f"Difficulty of the patch: {difficulty}")
        patches.append((view1, view2))
        difficulties.append(difficulty)

    # Neural Network Model : Efficent Net B0
    model = ES2FLModel().to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    print("Training Process of The Efficientnet-B0 Using Our Proposed Method Has Been Started")

    start_time = time.time()
    epochs = 1
    total_epochs = 1

    for epoch in range(1, total_epochs + 1):

      print(f"Training Epoch {epoch}/{total_epochs}")
      model = train_with_cumulative_curriculum(patches=patches,
        difficulties=difficulties,
        batch_size=64,
        model=model,
        optimizer=optimizer,
        device=device,
        epoch=epoch,
        total_epochs=total_epochs)

    print("The Efficientnet-B0 Has been Trained")

    end_time = time.time()
    elapsed_time = end_time - start_time
    minutes = int(elapsed_time // 60)
    seconds = int(elapsed_time % 60)
    print(f"Training completed in {minutes} minutes and {seconds} seconds")

    centers, labels = get_labeled_centers(gt, samples_per_class=5, num_classes=5)
    all_centers = [(i, j) for i in range(hsi.shape[0]) for j in range(hsi.shape[1]) if gt[i, j] > 0]

    patch_sizes = [32, 64]
    final_ensemble_probs = None
    total_weights = 0.0

    print("Starting The Second Phase of The Proposed Method: Feature Level and View Level Ensemble Strategy")

    num_bands = hsi.shape[2]
    band_mutual_info = np.zeros((num_bands, num_bands))

    for i in range(num_bands):
        for j in range(i+1, num_bands):
            band_mutual_info[i, j] = mutual_info_score(hsi[:, :, i].flatten(), hsi[:, :, j].flatten())
            band_mutual_info[j, i] = band_mutual_info[i, j]

    print("Mutual information between bands calculated.")

    for band_idx in range(num_bands):
        print(f"Creating Group for Band {band_idx + 1} of {num_bands}")

        mutual_info_scores = band_mutual_info[band_idx]
        sorted_band_indices = np.argsort(mutual_info_scores)[-20:]
        group_bands = [band_idx] + sorted_band_indices.tolist()
        print(f"Group {band_idx+1}: {len(group_bands)} Spectral Features")

        hsi_subset = hsi[:, :, group_bands]

        spectral_features = extract_raw_spectral_features(hsi_subset, centers)
        all_spectral_features = extract_raw_spectral_features(hsi_subset, all_centers)

        group_features_list = []
        group_all_features_list = []

        for ps in patch_sizes:
            print(f"View Level Ensemble: Patch Size {ps}")

            start_time = time.time()
            features = extract_features(model, hsi_subset, patch_size=ps, centers=centers, device=device)
            print(f"Deep Features of {ps} Patch Size: {features.shape[-1]} Features")
            end_time = time.time()
            minutes = int((end_time - start_time) // 60)
            seconds = int((end_time - start_time) % 60)
            print(f"Feature Extraction Process In Training Phase For Patch Size {ps} and Group {band_idx + 1} Has Been Completed in {minutes} minutes and {seconds} seconds")

            group_features_list.append(features)

            start_time = time.time()
            all_features = extract_features(model, hsi_subset, patch_size=ps, centers=all_centers, device=device)
            end_time = time.time()
            minutes = int((end_time - start_time) // 60)
            seconds = int((end_time - start_time) % 60)
            print(f"Feature Extraction Process In Test Phase For Patch Size {ps} and Group {band_idx + 1} Has Been Completed in {minutes} minutes and {seconds} seconds")

            group_all_features_list.append(all_features)

        # Combine features from different patch sizes and spectral features
        combined_features = np.concatenate(group_features_list + [spectral_features], axis=1)
        combined_all_features = np.concatenate(group_all_features_list + [all_spectral_features], axis=1)
        padded_combined_features = apply_padding_to_features(combined_features, patch=3)
        print(padded_combined_features.shape)
        padded_combined_all_features = apply_padding_to_features(combined_all_features, patch=3)
        print(padded_combined_all_features.shape)

        # Apply Local Graph-Based Fusion on the extracted features (combined_features and combined_all_features)
        fused_features_train = []
        fused_output_train = batch_feature_fusion(padded_combined_features, output_channels=40, k=4)
        fused_features_train = np.array(fused_output_train)
        print("Feature fusion for the train dataset has been completed.")

        print(f"Fused Deep Features for Train Set: {fused_features_train.shape[-1]} Features")

        NN = fused_features_train.shape[0]
        print(f"N : {NN} Train Samples")
        fused_features_train = fused_features_train.reshape(NN, -1)

        print(f"Fused Deep Features for Train Set After Reshape: {fused_features_train.shape[-1]} Features")

        fused_all_features_test = []
        fused_output_test = batch_feature_fusion(padded_combined_all_features, output_channels=40, k=4)
        fused_all_features_test = np.array(fused_output_test)
        print("Feature fusion for the test dataset has been completed.")
        # N = fused_all_features_test.shape[0]
        NN = fused_all_features_test.shape[0]
        print(f"N : {NN} Test Samples")
        fused_all_features_test = fused_all_features_test.reshape(NN, -1)



        # Train the SVM Classifier using the fused features
        start_time = time.time()
        clf = SVC(probability=True, kernel='rbf', C=1.0, gamma='scale', random_state=42)
        clf.fit(fused_features_train, labels)
        end_time = time.time()
        minutes = int((end_time - start_time) // 60)
        seconds = int((end_time - start_time) % 60)
        print(f"SVM Classifier Has Been Trained For Group {band_idx + 1} in {minutes} minutes and {seconds} seconds")

        probs = clf.predict_proba(fused_all_features_test)
        print(f"Test Phase For Group {band_idx + 1} Has Been Completed Using The SVM Classifier")

        true_labels = np.array([gt[i, j] for (i, j) in all_centers])
        pred = clf.predict(fused_all_features_test)
        acc = accuracy_score(true_labels, pred)
        print(f"[Group {band_idx + 1}/{num_bands}] Classification Accuracy: {acc:.4f}")

        weight = acc
        total_weights += weight

        if final_ensemble_probs is None:
            final_ensemble_probs = probs * weight
        else:
            final_ensemble_probs += probs * weight

    final_ensemble_probs /= total_weights
    final_pred = np.argmax(final_ensemble_probs, axis=1)

    classified_map = np.zeros(gt.shape, dtype=int)
    valid_idx = np.argwhere(gt > 0)
    for idx, (i, j) in enumerate(valid_idx):
        classified_map[i, j] = final_pred[idx]

    gt_flat = gt[gt > 0].flatten()
    oa = accuracy_score(gt_flat, final_pred) * 100
    conf_matrix = confusion_matrix(gt_flat, final_pred)
    report = classification_report(gt_flat, final_pred)

    print("Confusion Matrix:")
    print(conf_matrix)
    print("\nClassification Report:")
    print(report)
    print(f"Overall Accuracy (OA): {oa:.2f}%")

    plt.figure(figsize=(8, 6))
    plt.imshow(classified_map, cmap='jet')
    plt.colorbar()
    plt.title("Ensemble Classification Map (Patch Sizes: 32, 64, 128)")
    plt.show()


if __name__ == '__main__':
    main()

